In [5]:
from Bio import SeqIO
import pandas as pd
import numpy as np


chrdict = {}

for i, record in enumerate(SeqIO.parse("..\\dataset\\\GRCh37.primary_assembly.genome.fa", "fasta")):
    chrdict[record.id] = record.seq


# LE SE SEQUENZE DI BASI AZOTATE SONO MEMORIZZATE IN UN DIZIONARIO CHE USA COME CHIAVE "chr#" dove # indica
# il numero del cromosoma oppure X/Y/M (M non ci interessa perché é il DNA mitocondriale che non viene usato per questo caso)


# UA INIZIA INVECE LA PARTE DI LETTURA DEL FILE CONTENENTE I TSS


def extract_p(p):
    return p.split("@")[0]

def extract_gene_name(string):
    string =  string.split("@")[1]
    return string.split(",")[0]

v1 = np.vectorize(extract_p)
v2 = np.vectorize(extract_gene_name)




<>:8: SyntaxWarning: invalid escape sequence '\G'
<>:8: SyntaxWarning: invalid escape sequence '\G'
C:\Users\dany0\AppData\Local\Temp\ipykernel_14800\4216738839.py:8: SyntaxWarning: invalid escape sequence '\G'
  for i, record in enumerate(SeqIO.parse("..\\dataset\\\GRCh37.primary_assembly.genome.fa", "fasta")):


In [8]:
chrdict

{'chr1': Seq('NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN...NNN'),
 'chr10': Seq('NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN...NNN'),
 'chr11': Seq('NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN...NNN'),
 'chr12': Seq('NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN...NNN'),
 'chr13': Seq('NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN...NNN'),
 'chr14': Seq('NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN...NNN'),
 'chr15': Seq('NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN...NNN'),
 'chr16': Seq('NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN...NNN'),
 'chr17': Seq('AAGCTTCTCACCCTGTTCCTGCATAGATAATTGCATGACAATTGCCTTGTCCCT...GGT'),
 'chr18': Seq('NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN...NNN'),
 'chr19': Seq('NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN...NNN'),
 'chr2': Seq('NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN...NNN'),
 'chr20': Seq('NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN

In [10]:
df = pd.read_csv("..\\dataset\\TSS_human.bed", delimiter="\t", skiprows=1, names=["chromosome_name", "start", "end", "locus_id", "color", "strand", "start2", "end2", "RGB"])

df['p'] = v1(df['locus_id'].values)
df['gene_name'] = v2(df['locus_id'].values)

df

,chromosome_name,start,end,locus_id,color,strand,start2,end2,RGB,p,gene_name
0,chr10,100008587,100008589,"p1@CU680531,0.1352",-89.0,+,100008587,100008589,"211,211,211",p1,CU680531
1,chr10,100015362,100015397,"p2@LOXL4,0.1291",55.0,-,100015362,100015397,"211,211,211",p2,LOXL4
2,chr10,100017518,100017519,"p3@LOXL4,0.1842",13.0,-,100017518,100017519,"30,144,255",p3,LOXL4
3,chr10,100027943,100027958,"p1@LOXL4,0.2200",48.0,-,100027943,100027958,"30,144,255",p1,LOXL4
4,chr10,100174900,100174956,"p1@PYROXD2,0.2721",0.0,-,100174900,100174956,"60,179,113",p1,PYROXD2
...,...,...,...,...,...,...,...,...,...,...,...
1048119,chrY,9958814,9958815,"p@chrY:9958814..9958815,-,0.0551",0.0,-,9958814,9958815,"211,211,211",p,chrY:9958814..9958815
1048120,chrY,9959226,9959229,"p@chrY:9959226..9959229,-,0.0639",0.0,-,9959226,9959229,"211,211,211",p,chrY:9959226..9959229
1048121,chrY,9968102,9968103,"p@chrY:9968102..9968103,-,0.1091",0.0,-,9968102,9968103,"211,211,211",p,chrY:9968102..9968103
1048122,chrY,9985705,9985711,"p@chrY:9985705..9985711,-,0.0853",0.0,-,9985705,9985711,"211,211,211",p,chrY:9985705..9985711


In [13]:
meging=pd.read_csv("..\\dataset\\id_name_world.txt", sep="\t")

#merging su gene_name
df1 = pd.merge(df, meging, on="gene_name")
df1

,chromosome_name,start,end,locus_id,color,strand,start2,end2,RGB,p,gene_name,gene_id
0,chr10,100015362,100015397,"p2@LOXL4,0.1291",55.0,-,100015362,100015397,"211,211,211",p2,LOXL4,ENSG00000138131.4
1,chr10,100017518,100017519,"p3@LOXL4,0.1842",13.0,-,100017518,100017519,"30,144,255",p3,LOXL4,ENSG00000138131.4
2,chr10,100027943,100027958,"p1@LOXL4,0.2200",48.0,-,100027943,100027958,"30,144,255",p1,LOXL4,ENSG00000138131.4
3,chr10,100174900,100174956,"p1@PYROXD2,0.2721",0.0,-,100174900,100174956,"60,179,113",p1,PYROXD2,ENSG00000119943.13
4,chr10,100174957,100174982,"p2@PYROXD2,0.2448",0.0,-,100174957,100174982,"60,179,113",p2,PYROXD2,ENSG00000119943.13
...,...,...,...,...,...,...,...,...,...,...,...,...
254091,chrY,7142206,7142215,"p9@PRKY,0.2620",-120.0,+,7142206,7142215,"60,179,113",p9,PRKY,ENSG00000293442.1
254092,chrY,7200834,7200838,"p12@PRKY,0.0723",-232.0,+,7200834,7200838,"211,211,211",p12,PRKY,ENSG00000099725.15
254093,chrY,7200834,7200838,"p12@PRKY,0.0723",-232.0,+,7200834,7200838,"211,211,211",p12,PRKY,ENSG00000293442.1
254094,chrY,7200913,7200914,"p14@PRKY,0.0573",-156.0,+,7200913,7200914,"211,211,211",p14,PRKY,ENSG00000099725.15


In [22]:
patient=pd.read_csv("..\\dataset\\Dataset_median.csv", sep=",")
#merge su gene_id
df2 = pd.merge(df1, patient, on="gene_id")
df2

,chromosome_name,start,end,locus_id,color,strand,start2,end2,RGB,p,gene_name,gene_id,fpkm_median,tpm_median,fpkm_uq_median
0,chr10,100015362,100015397,"p2@LOXL4,0.1291",55.0,-,100015362,100015397,"211,211,211",p2,LOXL4,ENSG00000138131.4,1.800827,3.202559,1.977610
1,chr10,100017518,100017519,"p3@LOXL4,0.1842",13.0,-,100017518,100017519,"30,144,255",p3,LOXL4,ENSG00000138131.4,1.800827,3.202559,1.977610
2,chr10,100027943,100027958,"p1@LOXL4,0.2200",48.0,-,100027943,100027958,"30,144,255",p1,LOXL4,ENSG00000138131.4,1.800827,3.202559,1.977610
3,chr10,100174900,100174956,"p1@PYROXD2,0.2721",0.0,-,100174900,100174956,"60,179,113",p1,PYROXD2,ENSG00000119943.13,1.167486,2.608927,1.382999
4,chr10,100174957,100174982,"p2@PYROXD2,0.2448",0.0,-,100174957,100174982,"60,179,113",p2,PYROXD2,ENSG00000119943.13,1.167486,2.608927,1.382999
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
153925,chrY,2870826,2870844,"p2@ZFY-AS1,0.0950",-159.0,-,2870826,2870844,"211,211,211",p2,ZFY-AS1,ENSG00000233070.1,-7.643856,-7.643856,-7.643856
153926,chrY,6778505,6778534,"p6@TBL1Y,0.3422",-192.0,+,6778505,6778534,"60,179,113",p6,TBL1Y,ENSG00000092377.15,-7.643856,-7.643856,-7.643856
153927,chrY,6778592,6778601,"p7@TBL1Y,0.4173",-125.0,+,6778592,6778601,"60,179,113",p7,TBL1Y,ENSG00000092377.15,-7.643856,-7.643856,-7.643856
153928,chrY,6778602,6778616,"p4@TBL1Y,0.4601",-110.0,+,6778602,6778616,"60,179,113",p4,TBL1Y,ENSG00000092377.15,-7.643856,-7.643856,-7.643856


In [23]:
import re

def estrai_numero(s):
    match = re.match(r'p(\d+)', s)
    if match:
        return int(match.group(1))
    else:
        return None

# Applica la funzione alla colonna del DataFrame
df2['p_value'] = df2['p'].apply(estrai_numero)
df2

,chromosome_name,start,end,locus_id,color,strand,start2,end2,RGB,p,gene_name,gene_id,fpkm_median,tpm_median,fpkm_uq_median,p_value
0,chr10,100015362,100015397,"p2@LOXL4,0.1291",55.0,-,100015362,100015397,"211,211,211",p2,LOXL4,ENSG00000138131.4,1.800827,3.202559,1.977610,2
1,chr10,100017518,100017519,"p3@LOXL4,0.1842",13.0,-,100017518,100017519,"30,144,255",p3,LOXL4,ENSG00000138131.4,1.800827,3.202559,1.977610,3
2,chr10,100027943,100027958,"p1@LOXL4,0.2200",48.0,-,100027943,100027958,"30,144,255",p1,LOXL4,ENSG00000138131.4,1.800827,3.202559,1.977610,1
3,chr10,100174900,100174956,"p1@PYROXD2,0.2721",0.0,-,100174900,100174956,"60,179,113",p1,PYROXD2,ENSG00000119943.13,1.167486,2.608927,1.382999,1
4,chr10,100174957,100174982,"p2@PYROXD2,0.2448",0.0,-,100174957,100174982,"60,179,113",p2,PYROXD2,ENSG00000119943.13,1.167486,2.608927,1.382999,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
153925,chrY,2870826,2870844,"p2@ZFY-AS1,0.0950",-159.0,-,2870826,2870844,"211,211,211",p2,ZFY-AS1,ENSG00000233070.1,-7.643856,-7.643856,-7.643856,2
153926,chrY,6778505,6778534,"p6@TBL1Y,0.3422",-192.0,+,6778505,6778534,"60,179,113",p6,TBL1Y,ENSG00000092377.15,-7.643856,-7.643856,-7.643856,6
153927,chrY,6778592,6778601,"p7@TBL1Y,0.4173",-125.0,+,6778592,6778601,"60,179,113",p7,TBL1Y,ENSG00000092377.15,-7.643856,-7.643856,-7.643856,7
153928,chrY,6778602,6778616,"p4@TBL1Y,0.4601",-110.0,+,6778602,6778616,"60,179,113",p4,TBL1Y,ENSG00000092377.15,-7.643856,-7.643856,-7.643856,4


In [28]:

df3 = df2.loc[df2.groupby("gene_name")["p_value"].idxmin()]



df3.value_counts("p")

p
p1    14075
p2       86
p3       14
p4       10
p5        5
p6        3
p7        2
p8        2
p9        1
Name: count, dtype: int64

In [27]:
df4 = df2.loc[df2.groupby("gene_id")["p_value"].idxmin()]
df4

,chromosome_name,start,end,locus_id,color,strand,start2,end2,RGB,p,gene_name,gene_id,fpkm_median,tpm_median,fpkm_uq_median,p_value
153866,chrX,99839925,99839938,"p1@TNMD,0.1353",126.0,+,99839925,99839938,"211,211,211",p1,TNMD,ENSG00000000005.6,-3.925769,-2.475535,-3.733124,1
38903,chr1,169863016,169863109,"p1@SCYL3,0.2639",0.0,-,169863016,169863109,"60,179,113",p1,SCYL3,ENSG00000000457.14,0.873892,2.396543,1.063089,1
42198,chr1,27961629,27961665,"p1@FGR,0.5037",0.0,-,27961629,27961665,"60,179,113",p1,FGR,ENSG00000000938.13,0.612022,2.081715,0.757621,1
71796,chr6,143832820,143832875,"p1@FUCA2,0.4000",0.0,-,143832820,143832875,"60,179,113",p1,FUCA2,ENSG00000001036.14,3.827545,5.333456,4.001946,1
74636,chr6,53409890,53409906,"p1@GCLC,0.4228",-4.0,-,53409890,53409906,"60,179,113",p1,GCLC,ENSG00000001084.13,1.509544,2.956224,1.694702,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86592,chrX,101967366,101967391,"p1@ARMCX5-GPRASP2,p1@GPRASP2,0.6351",0.0,+,101967366,101967391,"60,179,113",p1,ARMCX5-GPRASP2,ENSG00000286237.1,-7.643856,-7.643856,-7.643856,1
65961,chr4,89513578,89513670,"p1@HERC3,0.3732",0.0,+,89513578,89513670,"60,179,113",p1,HERC3,ENSG00000287542.1,-7.643856,-7.643856,-7.643856,1
82689,chr8,67579807,67579860,"p1@C8orf44,p1@SGK3,0.4647",0.0,+,67579807,67579860,"60,179,113",p1,C8orf44,ENSG00000288596.2,0.995448,2.458933,1.200316,1
148900,chr8,53852965,53852990,"p1@NPBWR1,0.2996",497.0,+,53852965,53852990,"60,179,113",p1,NPBWR1,ENSG00000288611.1,-6.687800,-5.779918,-6.546245,1
